# CCA175 Practice Exam
* Exam    : CCA175
* Title   : CCA Spark and Hadoop Developer Exam
* Vendor  : Cloudera
* Version : V12.35

IT Certification Guaranteed, The Easy Way!

## NO.39 CORRECT TEXT
### Problem Scenario 80 : 

You have been given MySQL DB with following details.

|||
|-|-|
|user     | retail_dba |
|password | cloudera   |
|database | retail_db  |
|table    | retail_db.products |
|jdbc URL |  jdbc:mysql://quickstart:3306/retail_db|

Columns of products table : 

`(product_id | product_category_id | product_name | product_description | product_price | product_image )`

Please accomplish following activities.
1. Copy `retaildb.products` table to hdfs in a directory p39
2. Now sort the products data sorted by product price per category, use `product_category_id` column to group by category

Solution from book is wird

See the explanation for Step by Step Solution and configuration.
Explanation:
Solution :
Step 1 : Import Single table .
sqoop import --connect jdbc:mysql://quickstart:3306/retail_db -username=retail_dba -
password=cloudera -table=products --target-dir=p93
Note : Please check you dont have space between before or after '=' sign. Sqoop uses the
MapReduce framework to copy data from RDBMS to hdfs

Step 2 : Step 2 : Read the data from one of the partition, created using above command, hadoop fs -
cat p93_products/part-m-00000
Step 3 : Load this directory as RDD using Spark and Python (Open pyspark terminal and do following}.
productsRDD = sc.textFile(Mp93_products")
Step 4 : Filter empty prices, if exists

--filter out empty prices lines
Nonempty_lines = productsRDD.filter(lambda x: len(x.split(",")[4]) > 0)
                          
Step 5 : Create data set like (categroyld, (id,name,price)
mappedRDD = nonempty_lines.map(
  lambda line: 
  (   
    line.split(",")[1], 
    ( line.split(",")[0], line.split(",")[2], float(line.split(",")[4]) ) 
  )
) 
for line in mappedRDD.collect(): print(line)
                               
Step 6 : Now groupBy the all records based on categoryld, which a key on mappedRDD it will produce
output like (categoryld, iterable of all lines for a key/categoryld) 
groupByCategroyld =mappedRDD.groupByKey() 
for line in groupByCategroyld.collect():print(line)
                               
                               
steps 7 and 8 give ERRORS                               
                               
step 7 : Now sort the data in each category based on price in ascending order.
-- sorted is a function to sort an iterable, we can also specify, what would be the Key on which we
want to sort in this case we have price on which it needs to be sorted.
groupByCategroyld.map(lambda tuple: 
                      sorted( 
                              tuple[1], 
                              key=lambda tupleValue:tupleValue[2] )
                     )
                 .take(5)
                               
Step 8 : Now sort the data in each category based on price in descending order.
-- sorted is a function to sort an iterable, we can also specify, what would be the Key on which we
want to sort in this case we have price which it needs to be sorted.
on groupByCategroyld.map(lambda tuple: sorted(tuple[1], key=lambda tupleValue:
tupleValue[2] , reverse=True)).take(5)

#### Answer.
AAT

import data from mySQL database to hdfs

In [6]:
sqoop import
--connect  jdbc:mysql://quickstart:3306/retail_db
--username=retail_dba
--password=cloudera
--table    products
--fields-terminated-by '\t'
--target-dir /user/cloudera/p39


open pyspark and load some libraries

In [8]:
import pyspark.sql
import pyspark.sql.functions
from pyspark.sql import SQLContext, Row

In [9]:
def p2(t):
  print( 'key:{}'.format( t[0] ) )
  for i in t[1]: print( '\t{}'.format( i ) )

load data into an rdd of tuples

In [11]:
#product_id | product_category_id | product_name | product_description | product_price
# category, ( id, name, price )

tuples = sc.textFile( '/user/cloudera/p39' ).map( lambda line: line.split( '\t' ) ).map( lambda a: ( int(a[1]), int(a[0]), a[2], float(a[4] ) ) )

create rdd of Data Rows

In [13]:
rows = tuples.map( lambda a: Row( product_category_id = int( a[0] ), product_id= int( a[1] ), product_name = a[2], product_price = float( a[3] ) ) )


create dataframe `products` and register it to use it in sql queries.

In [15]:
products = rows.toDF()
products.registerTempTable( 'products' )

In [16]:
# sort by price in ascending
df1 = sqlContext.sql( """
SELECT product_category_id, product_id, product_name, product_price 
FROM products 
ORDER BY product_category_id, product_price
"""
)

# group
g1 = df1.rdd.map( lambda a: ( int(a['product_category_id']), ( int(a['product_id']), a['product_name'], float(a['product_price']) ) ) ).groupByKey()

# print results
for i in g1.take(5): p2( i )

In [17]:
# sort by price in descending order
df2 = sqlContext.sql( """
SELECT product_category_id, product_id, product_name, product_price 
FROM products 
ORDER BY product_category_id, product_price DESC
""" )

# group
g2 = df2.rdd.map( lambda a: ( int(a['product_category_id']), ( int(a['product_id']), a['product_name'], float(a['product_price']) ) ) ).groupByKey()

# print results
for i in g2.take(5): p2( i )

end
---